In [0]:
pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import nltk
import math
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')
nltk.download('stopwords')
stpwords =  set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
raw_data=sc.textFile("dbfs:/FileStore/shared_uploads/sxs230164@utdallas.edu/imdb_labelled.txt")
data = raw_data.map(lambda x: (x.split("\t")[1],x.split("\t")[0]))
data.take(10)

Out[86]: [('0',
  'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  '),
 ('0',
  'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  '),
 ('0',
  'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  '),
 ('0', 'Very little music or anything to speak of.  '),
 ('1',
  'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  '),
 ('0',
  "The rest of the movie lacks art, charm, meaning... If it's about emptiness, it works I guess because it's empty.  "),
 ('0', 'Wasted two hours.  '),
 ('1',
  'Saw the movie today and thought it was a good effort, good messages for kids.  '),
 ('0', 'A bit predictable.  '),
 ('1', 'Loved the casting of Jimmy Buffet as the science teacher.  ')]

In [0]:
def stopword_removal(word):
    nostopwords = []
    for i in word:
        if i.lower() not in stpwords:
            nostopwords.append(i)
    return nostopwords
 
def punctuation_removal(word):
    pset = '''!()[]{};:"\,<>./?@#$%^&*_~'''
    for i in word:
        if i in pset:
            word = word.replace(i, "")
    return word
 
def lemmatize(words):
    lemm = WordNetLemmatizer()
    lem_tokens = [lemm.lemmatize(x) for x in words]
    return lem_tokens

In [0]:
cleandata1 = data.mapValues(lambda x: punctuation_removal(x))
cleandata2 = cleandata1.mapValues(lambda x: re.sub('[^a-zA-Z]',' ',x))
cleandata3 = cleandata2.mapValues(lambda x: x.lower()).mapValues(lambda x:nltk.word_tokenize(x))
cleandata4 = cleandata3.mapValues(lambda x: stopword_removal(x))
finaldata = cleandata4.mapValues(lambda x: lemmatize(x)).map(lambda x:(x[1],x[0]))
finaldata.take(10)
#cleandata1 removes all punctuations from data
#cleandata2 removes all digits from cleandata1 using sub method
#cleandata3 coverts cleandata2 into lower case and performs word tokenization
#cleadata4 removes all stop words from word tokenization data(cleandata3)
#finaldata lemmatizes the list of review words and finally the data is reversed as (list of words, label ) 

Out[88]: [(['slow',
   'moving',
   'aimless',
   'movie',
   'distressed',
   'drifting',
   'young',
   'man'],
  '0'),
 (['sure',
   'lost',
   'flat',
   'character',
   'audience',
   'nearly',
   'half',
   'walked'],
  '0'),
 (['attempting',
   'artiness',
   'black',
   'white',
   'clever',
   'camera',
   'angle',
   'movie',
   'disappointed',
   'became',
   'even',
   'ridiculous',
   'acting',
   'poor',
   'plot',
   'line',
   'almost',
   'non',
   'existent'],
  '0'),
 (['little', 'music', 'anything', 'speak'], '0'),
 (['best',
   'scene',
   'movie',
   'gerardo',
   'trying',
   'find',
   'song',
   'keep',
   'running',
   'head'],
  '1'),
 (['rest',
   'movie',
   'lack',
   'art',
   'charm',
   'meaning',
   'emptiness',
   'work',
   'guess',
   'empty'],
  '0'),
 (['wasted', 'two', 'hour'], '0'),
 (['saw',
   'movie',
   'today',
   'thought',
   'good',
   'effort',
   'good',
   'message',
   'kid'],
  '1'),
 (['bit', 'predictable'], '0'),
 (['loved', 'cast

In [0]:
train, test = finaldata.randomSplit([0.8, 0.2])

In [0]:
train_count = train.count()
test_count = test.count()

In [0]:
positives = train.filter(lambda x: x[1] == "1").count()
positivecount = train.filter(lambda x: x[1] == "1").flatMap(lambda x: x[0]).count()

positive_log_probablity = math.log(positives/ (train_count)) #used while testing phase
positive_log_probablity

Out[91]: -0.6784410331702498

In [0]:
negatives = train.filter(lambda x: x[1] == "0").count()
negativecount = train.filter(lambda x: x[1] == "0").flatMap(lambda x: x[0]).count()
# negativecount
negative_log_probablity = math.log(negatives/ (train_count))

In [0]:
differentwords = train.flatMap(lambda x: x[0]).distinct().count()

In [0]:
alpha = 1
positive_density = positivecount + ((differentwords+1)*alpha)
 
positive_train = train.filter(lambda x: x[1] == "1").flatMap(lambda x: x[0]).map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y).mapValues(lambda x: (x+alpha)/positive_density) 

In [0]:
alpha = 1
negative_density = negativecount + ((differentwords+1)*alpha)
 
negative_train = train.filter(lambda x: x[1] == "0").flatMap(lambda x: x[0]).map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y).mapValues(lambda x: (x+alpha)/negative_density) 
negative_train.sortBy(lambda x: -x[1]).take(100)

Out[95]: [('movie', 0.01818181818181818),
 ('film', 0.010671936758893281),
 ('bad', 0.009881422924901186),
 ('even', 0.005731225296442688),
 ('one', 0.005731225296442688),
 ('plot', 0.0047430830039525695),
 ('like', 0.004545454545454545),
 ('time', 0.004347826086956522),
 ('really', 0.004150197628458498),
 ('acting', 0.004150197628458498),
 ('character', 0.0037549407114624506),
 ('script', 0.0035573122529644267),
 ('scene', 0.0033596837944664033),
 ('make', 0.0031620553359683794),
 ('would', 0.0031620553359683794),
 ('story', 0.0031620553359683794),
 ('line', 0.002766798418972332),
 ('good', 0.002766798418972332),
 ('see', 0.002766798418972332),
 ('work', 0.002569169960474308),
 ('ever', 0.002569169960474308),
 ('seen', 0.002569169960474308),
 ('stupid', 0.002569169960474308),
 ('little', 0.002569169960474308),
 ('much', 0.002569169960474308),
 ('made', 0.002569169960474308),
 ('thing', 0.002569169960474308),
 ('show', 0.002569169960474308),
 ('worst', 0.0023715415019762848),
 ('could'

In [0]:
test_index = test.zipWithIndex()
test_data = test_index.map(lambda x: (x[1],x[0][0]))
test_label = test_index.map(lambda x: (x[1],x[0][1]))
split_test_words = test_data.flatMap(lambda x:((x[0],(i)) for i in x[1])).map(lambda x: (x[1],x[0]))
split_test_words.take(10)

Out[96]: [('attempting', 0),
 ('artiness', 0),
 ('black', 0),
 ('white', 0),
 ('clever', 0),
 ('camera', 0),
 ('angle', 0),
 ('movie', 0),
 ('disappointed', 0),
 ('became', 0)]

In [0]:
positive_probability_test = split_test_words.leftOuterJoin(positive_train).map(lambda x: (x[0],(x[1][0],1/positive_density if x[1][1] is None else x[1][1])))
positive_probability_log_test = positive_probability_test.map(lambda x: (x[1][0],x[1][1])).map(lambda x: (x[0],math.log(x[1]))).reduceByKey(lambda x,y: x+y).map(lambda x: (x[0],x[1]+positive_log_probablity))

In [0]:
neg_probability_test = split_test_words.leftOuterJoin(negative_train).map(lambda x: (x[0],(x[1][0],1/negative_density if x[1][1] is None else x[1][1])))
neg_probability_log_test = neg_probability_test.map(lambda x: (x[1][0],x[1][1])).map(lambda x: (x[0],math.log(x[1]))).reduceByKey(lambda x,y: x+y).map(lambda x: (x[0],x[1]+negative_log_probablity))

In [0]:
test_probability = neg_probability_log_test.fullOuterJoin(positive_probability_log_test)
predict_test = test_probability.map(lambda x: (x[0],"0" if x[1][0] > x[1][1] else "1"))
original_predict = predict_test.join(test_label)

In [0]:
org_count = original_predict.filter(lambda x: x[1][1] == x[1][0]).count() 
accuracy =(org_count)/(test_count)
prior_neg = negatives/ (train_count)
prior_pos = positives/ (train_count)

In [0]:
print("Accuracy of the model ", accuracy)
print("prior probability of positive reveiws ", prior_pos)
print("prior probability of negative reviews ", prior_neg)

Accuracy of the model  0.8052631578947368
prior probability of positive reveiws  0.5074074074074074
prior probability of negative reviews  0.4925925925925926
